# Özellik Piramidi Ağları Türkçe Anlatım (FPN)


Özellik piramidi ağları, genellikle görüntü işlemede kullanılan bir yapay sinir ağı tasarımıdır. Bu ağlar, girdi resmin özelliklerini birden fazla katman ve farklı ölçeklerde öğrenmek için tasarlanmıştır. Özellik piramidi ağı genellikle şöyle yapılandırılır:


## Giriş Katmanı

Özellik piramidi ağlarının giriş katmanı, ağın işlem yapacağı girdi resmini alır. Bu katman, resmin özelliklerini algılamak için kullanılan ilk katmandır. Giriş katmanı, resmin özelliklerini ön işlem yaparak daha işleme uygun hale getirir. Ön işlemler arasında resmin boyutlarını azaltma, renk düzenleme, histogram eşitleme gibi işlemler yer alır.

Giriş katmanı ayrıca, resmin özelliklerini küçülterek, daha hızlı işleme olanak tanır. Örneğin, giriş katmanı resmi birkaç parçaya bölerek, her parçayı ayrı ayrı işleyebilir. Bu sayede ağ, resmin özelliklerini daha hızlı öğrenebilir.

Giriş katmanı, ağın işlem yapacağı girdi resmin özelliklerini ön işlem yaparak daha işleme uygun hale getirir, ve girdi resmin özelliklerini küçülterek daha hızlı işleme olanak tanır. Bu sayede ağ, konsept katmanlarının öğrenmesi için daha iyi bir zemin oluşur.

Aşağıdaki örnekte girdi resmin ön işlem yaparak boyutlarını azaltma ve renk düzenleme işlemlerini PyTorch API kullanarak yapacağım. 

In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Girdi resmini okuma
image = Image.open("input.jpg")
# Girdi resmini ön işlem yapma
processed_image = transform(image)

Bu kod örneğinde, transforms.Resize(256) ile girdi resmini 256x256 boyutuna küçültüyorum. Daha sonra transforms.CenterCrop(224) ile resmi ortasından 224x224 boyutunda bir kesit alıyorum. Daha sonra transforms.ToTensor() ile PIL image tipinden Tensor tipine dönüştürüyorum. Son olarak transforms.Normalize ile resmi normalize ediyorum. Bu işlemler ile resmin özellikleri daha işleme uygun hale getirilir. Bu işlemler giriş katmanının yapacağı ön işlemlerdir.

Pytorch torchvision.transforms modülü resmi işlemek için kullanabileceğimiz birçok fonksiyon sunmaktadır. Bu örnekte sadece birkaç ön işlem yaptım ama gerçek uygulamalarda daha fazla ön işlem yapılabilir.

## Konsept katmanları

Özellik piramidi ağlarının konsept katmanları, ağın girdi resmin daha yüksek seviyede özelliklerini öğrenmesini sağlar. Bu katmanlar, girdi resmin özelliklerini daha yinelemeli bir şekilde öğrenir ve daha ileri seviyede özellikleri öğrenir. Bu katmanlar arasındaki her bir katman, önceki katmanın özelliklerini kullanarak daha ileri seviyede özellikleri öğrenir.

Konsept katmanları, genellikle birden fazla katman olarak yapılandırılır. Her katman, önceki katmanın özelliklerini kullanarak daha ileri seviyede özellikleri öğrenir. Bu sayede, ağ, girdi resmin farklı ölçeklerdeki özelliklerini öğrenir ve daha doğru tahmin yapar.

Konsept katmanlarının türevleri arasında Smooth katmanlar ve Lateral katmanlar yer alır.


#### Smooth Katmanlar


Smooth katmanlar, önceki katmanın özelliklerini düzgün bir şekilde devam ettirerek daha yüksek seviyede özellikleri öğrenir. Bu katmanlar, özellikleri daha az parçacık hale getirerek daha az spektrumda özellikleri öğrenir. Bu sayede ağ, resmin daha genel özelliklerini öğrenir.


Bu amaçla kullanabileceğimiz birçok fonksiyon vardır ama burada `nn.Conv2d` ve `nn.MaxPool2d` kullanacağım. Örnek kod PyTorch API kullanarak yazılmıştır:

In [ ]:
import torch.nn as nn

class SmoothLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SmoothLayer, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.pool(x)
        return x

Bu kod, PyTorch API kullanarak smooth katmanını oluşturan bir sınıf tanımlar. Bu sınıf, girdi resmin özelliklerini, `nn.Conv2d()` katmanı aracılığıyla konvolüsyon yaparak öğrenir ve daha sonra `nn.MaxPool2d()` katmanı aracılığıyla max pooling yaparak özellikleri daha az parçacık hale getirir. Böylece, ağ, resmin daha genel özelliklerini öğrenir.

### Lateral Katmanlar


Lateral katmanlar ise, önceki katmanın özelliklerini daha yinelemeli bir şekilde öğrenir. Bu katmanlar, özellikleri daha parçacık hale getirerek daha fazla spektrumda özellikleri öğrenir. Bu sayede ağ, resmin daha detaylı özelliklerini öğrenir.


Bu amaçla, lateral katman olarak konvolüsyon katmanının stride'ını arttırarak veya pooling katmanının pooling boyutunu küçülterek yerine getirmesini kullanabilirsiniz. Örnek olarak, PyTorch API kullanarak lateral katmanı oluşturmak için aşağıdaki kodu verebilirim:

In [ ]:
import torch.nn as nn

class LateralLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(LateralLayer, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = self.conv(x)
        return x


Bu katmanlar arasındaki kombinasyon sayesinde ağ hem daha genel özellikleri öğrenir hem de daha detaylı özellikleri öğrenir. Bu sayede ağ daha doğru tahmin yapar. Özellik piramit ağlarının konsept katmanları, genellikle konvolüsyon katmanları veya pooling katmanları olarak yapılandırılır.